In [ ]:
import os
os.chdir("../")

In [ ]:
import numpy as np
import torch

In [ ]:
from s3ts.data.har_datasets import HARTHDataset
from s3ts.data.methods import *
from s3ts.data.dfdataset import LDFDataset, DFDataset
from s3ts.data.label_mappings import harth_label_mapping
from s3ts.api.nets.methods import create_model_from_DM, train_model

from torchvision.transforms import Normalize

In [ ]:
ds = HARTHDataset("./datasets/HARTH/", wsize=48, normalize=True, label_mapping=harth_label_mapping)
ds.indices = ds.indices[ds.indices <= ds.subject_indices[2]] ## only first 2
ds.splits = ds.splits[ds.splits <= ds.subject_indices[2]]
print(len(ds))

In [ ]:
meds = sts_medoids(ds, n=200)

dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None, ram=False)

In [ ]:
DM = []

np.random.seed(42)
for i in np.random.choice(np.arange(len(dfds)), 500):
    dm, _, _ = dfds[i]
    DM.append(dm)

DM = torch.stack(DM)

dm_transform = Normalize(mean=DM.mean(dim=[0, 2, 3]), std=DM.std(dim=[0, 2, 3]))
dfds.dm_transform = dm_transform

In [ ]:
data_split = split_by_test_subject(ds, 1)

dm = LDFDataset(dfds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8, reduce_train_imbalance=True)

In [ ]:
print(len(dm.ds_train))
print(len(dm.ds_val))
print(len(dm.ds_test))

In [ ]:
model = create_model_from_DM(dm, name=None, 
        dsrc="img", arch="cnn", task="cls", dec_arch="mlp", dec_layers=1)

In [ ]:
model, data = train_model(dm, model, max_epochs=2)
print(data)